In [20]:
import os
import math
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from data_load import load_dataset

In [21]:
series,times = load_dataset('data_h')
series = series.reshape(-1,1) #array bidimensional para utilizar luego el MinMaxScaler

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0.1,1)) # los valores de la loss function son menoros con este rango que con (0,1)
series_scaled = scaler.fit_transform(series)

In [22]:
train_percent = 0.7
val_percent = 0.2
test_percent = 0.1

train_size = int(len(series) * train_percent)
val_size = int(len(series) * val_percent)

x_train = series[:train_size]
x_train_scaled = series_scaled[:train_size]
time_train = times[:train_size]

x_val = series[train_size:train_size + val_size]
x_val_scaled = series_scaled[train_size:train_size + val_size]
time_val = times[train_size:train_size + val_size]

x_test = series[train_size + val_size:]
x_test_scaled = series_scaled[train_size + val_size:]
time_test = times[train_size + val_size:]

Vamos a considerar los 3 mejores modelos: 

* Bidirectional_LSTM_alt ws=96 bs=16
* LSTM_Stacked_3_alt ws=96 bs=16
* LSTM_Stacked_3 ws=96 bs=32

In [23]:
# Definir el nombre y la ruta del archivo del modelo guardado
model_name = 'LSTM_stacked_3_ws96_bs32.keras'
checkpoint_dir = 'Results/train_LSTM_stacked_3_ws96_bs32_results'
checkpoint_filepath = os.path.join(checkpoint_dir, model_name)

model = tf.keras.models.load_model(checkpoint_filepath, compile=False)
model.summary()

Model: "sequential_34"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_73 (LSTM)                  │ (None, 96, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_74 (LSTM)                  │ (None, 96, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_75 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 95,489 (373.00 KB)

 Trainable params: 95,489 (373.00 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
window_size = 24 * 4 # 4 days
num_features = 1
batch_size = 32

In [34]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

train_generator = TimeseriesGenerator(x_train_scaled, 
                                      x_train_scaled, # Esta bien utilizar la demanda como entrada y como objetivo a la vez ya que es una prediccion univariada
                                      length=window_size,
                                      batch_size=batch_size)

validation_generator = TimeseriesGenerator(x_val_scaled,
                                           x_val_scaled,
                                           length=window_size,
                                           batch_size=batch_size)

# normalizamos tambien los datos de test ya que el modelo estara entrenado con datos normalizados por lo que no sera muy consistente realizarlo sin normalizar
test_generator = TimeseriesGenerator(x_test_scaled,
                                           x_test_scaled,
                                           length=window_size,
                                           batch_size=batch_size)

In [35]:
from models import predict

train_predict = predict(model,
                        train_generator,
                        scaler)

val_predict = predict(model,
                       validation_generator,
                       scaler)

test_predict = predict(model,
                       test_generator,
                       scaler)

  5/189 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step

c:\Users\daviddpp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step
 8/52 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step

c:\Users\daviddpp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
 7/25 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step

c:\Users\daviddpp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step


In [36]:
errors_abs_train = np.abs(x_train[window_size:,0] - train_predict[:, 0])

min_abs_error = np.min(errors_abs_train)
q1_abs_error = np.percentile(errors_abs_train, 25)
median_abs_error = np.median(errors_abs_train)
mean_abs_error = np.mean(errors_abs_train)
q3_abs_error = np.percentile(errors_abs_train, 75)
max_abs_error = np.max(errors_abs_train)

print("Estadísticas de Errores Absolutos:")
print(f"Min.    {min_abs_error:.3f}")
print(f"1st Qu. {q1_abs_error:.3f}")
print(f"Median  {median_abs_error:.3f}")
print(f"Mean    {mean_abs_error:.3f}")
print(f"3rd Qu. {q3_abs_error:.3f}")
print(f"Max.    {max_abs_error:.3f}")

Estadísticas de Errores Absolutos:
Min.    0.002
1st Qu. 9.174
Median  20.988
Mean    30.174
3rd Qu. 43.435
Max.    332.234


In [37]:
errors_abs_test = np.abs(x_test[window_size:,0] - test_predict[:, 0])

min_abs_error = np.min(errors_abs_test)
q1_abs_error = np.percentile(errors_abs_test, 25)
median_abs_error = np.median(errors_abs_test)
mean_abs_error = np.mean(errors_abs_test)
q3_abs_error = np.percentile(errors_abs_test, 75)
max_abs_error = np.max(errors_abs_test)

print("Estadísticas de Errores Absolutos:")
print(f"Min.    {min_abs_error:.3f}")
print(f"1st Qu. {q1_abs_error:.3f}")
print(f"Median  {median_abs_error:.3f}")
print(f"Mean    {mean_abs_error:.3f}")
print(f"3rd Qu. {q3_abs_error:.3f}")
print(f"Max.    {max_abs_error:.3f}")

Estadísticas de Errores Absolutos:
Min.    0.093
1st Qu. 13.058
Median  30.369
Mean    39.809
3rd Qu. 57.238
Max.    225.939


In [31]:
errors_mape_train = errors_abs_train / x_train

min_abs_error = np.min(errors_mape_train)
q1_abs_error = np.percentile(errors_mape_train, 25)
median_abs_error = np.median(errors_mape_train)
mean_abs_error = np.mean(errors_mape_train)
q3_abs_error = np.percentile(errors_mape_train, 75)
max_abs_error = np.max(errors_mape_train)

print("Estadísticas de Errores Absolutos:")
print(f"Min.    {min_abs_error:.3f}")
print(f"1st Qu. {q1_abs_error:.3f}")
print(f"Median  {median_abs_error:.3f}")
print(f"Mean    {mean_abs_error:.3f}")
print(f"3rd Qu. {q3_abs_error:.3f}")
print(f"Max.    {max_abs_error:.3f}")

C:\Users\daviddpp\AppData\Local\Temp\ipykernel_36944\1210231178.py:1: RuntimeWarning: divide by zero encountered in divide
  errors_mape_train = errors_abs_train / x_train


Estadísticas de Errores Absolutos:
Min.    0.000
1st Qu. 0.069
Median  0.176
Mean    inf
3rd Qu. 0.400
Max.    inf


In [32]:
errors_mape_test = errors_abs_test / x_test

min_abs_error = np.min(errors_mape_test)
q1_abs_error = np.percentile(errors_mape_test, 25)
median_abs_error = np.median(errors_mape_test)
mean_abs_error = np.mean(errors_mape_test)
q3_abs_error = np.percentile(errors_mape_test, 75)
max_abs_error = np.max(errors_mape_test)

print("Estadísticas de Errores Absolutos:")
print(f"Min.    {min_abs_error:.3f}")
print(f"1st Qu. {q1_abs_error:.3f}")
print(f"Median  {median_abs_error:.3f}")
print(f"Mean    {mean_abs_error:.3f}")
print(f"3rd Qu. {q3_abs_error:.3f}")
print(f"Max.    {max_abs_error:.3f}")

Estadísticas de Errores Absolutos:
Min.    0.000
1st Qu. 0.085
Median  0.209
Mean    inf
3rd Qu. 0.451
Max.    inf


C:\Users\daviddpp\AppData\Local\Temp\ipykernel_36944\435371747.py:1: RuntimeWarning: divide by zero encountered in divide
  errors_mape_test = errors_abs_test / x_test
